In [1]:
import pandas as pd
import numpy as np
from montecarlo.montecarlo import Die, Game, Analyzer

sides = np.array([1,2,3,5])
d = Die(sides)

/home/hrn4ch/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
type(sides)

numpy.ndarray

In [3]:
isinstance(sides, np.ndarray)

True

In [4]:
df = pd.DataFrame({'sides':sides, 'weights':[1]*len(sides)})
df = df.set_index('sides')

In [5]:
df.sample(3, replace=True, weights=df['weights']).index.to_list()

[1, 2, 1]

In [7]:
df['weights'][5]

1

In [8]:
d.change_weight(5, 4)
d.state()

,weights
sides,
1,1
2,1
3,1
5,4


In [19]:
x = np.array(d.roll(100))
np.unique(x, return_counts=True)

(array([1, 2, 3, 5]), array([12, 15, 13, 60]))

In [14]:
df.sample(5, replace=True, weights=df['weights'])

,weights
sides,
1,1
2,1
3,1
5,1
2,1


In [12]:
x = pd.DataFrame()


In [11]:
d1 = Die(np.array([1,2,3]))
d2 = Die(np.array([1,2,3]))
d3 = Die(np.array([1,2,3]))

dice = [d1,d2,d3]
g = Game(dice)

In [12]:
g.play(4)
y = g.last_play()
y

,dice-0,dice-1,dice-2
roll,,,
0,1,3,3
1,2,2,2
2,3,2,1
3,3,1,2


In [14]:
x =g.last_play(True)
x.value_counts()

outcome
2          5
3          4
1          3
Name: count, dtype: int64

In [16]:
isinstance(g, Game)

True

In [9]:
type(y.index)

pandas.core.indexes.base.Index

In [43]:
df = g.last_play()
df['roll'] = df.index
df = pd.melt(df, id_vars = ['roll'], 
        value_vars = [i for i in range(len(df.columns)-1)])
df = df.rename({'variable': 'dice', 'value':'outcome'}, axis=1)
df = df.set_index(['roll', 'dice'])

,,outcome
roll,dice,
0,0,3
1,0,1
2,0,1
3,0,1
0,1,1
1,1,2
2,1,3
3,1,1
0,2,1


In [108]:
d1.change_weight(1, 1)
d2.change_weight(1, 1)
d3.change_weight(1, 1)

In [115]:
g.play(8)
df = g.last_play()
df

,dice-0,dice-1,dice-2
roll,,,
0,2,2,3
1,1,2,2
2,3,2,2
3,2,2,1
4,1,2,3
5,1,2,2
6,3,2,3
7,2,2,1


In [64]:
len(df['dice-0'].unique())

1

In [40]:
def jackpot(df):
    count = 0
    for c in df.columns:
        if len(df[c].unique())==1:
            count+=1
    return count
jackpot(df)

1

In [65]:
count = 0
for i in df.index:
    print(df.iloc[i].unique())
    if len(df.iloc[i].unique())==1:
        count+=1
count

[1]
[1 2]
[1 2]
[1]


2

In [73]:
wombocombos = pd.DataFrame(df.value_counts())
wombocombos.reset_index()

,dice-0,dice-1,dice-2,count
0,1,1,1,2
1,1,1,2,1
2,1,2,1,1


In [142]:
df

,dice-0,dice-1,dice-2
roll,,,
0,2,2,3
1,1,2,2
2,3,2,2
3,2,2,1
4,1,2,3
5,1,2,2
6,3,2,3
7,2,2,1


In [145]:
pd.DataFrame(np.sort(df.values, axis=1)).groupby([0,1,2]).value_counts()

0  1  2
1  2  2    4
      3    1
2  2  3    2
   3  3    1
Name: count, dtype: int64

In [141]:
combo = pd.DataFrame(np.sort(df.values, axis=1)).groupby([i for i in range(len(df.columns))]).value_counts()
combodf = pd.DataFrame(combo)
combodf

count
0 1 2       
1 2 2      4
    3      1
2 2 3      2
  3 3      1

count
0      1      2            
dice-1 dice-2 dice-2      4
              3           1
dice-2 dice-2 3           2
       3      3           1

In [2]:
d11 = Die(np.array([1,2,3]))
d21 = Die(np.array([1,2,3]))
d31 = Die(np.array([1,2,3]))

dice1 = [d11,d21,d31]
g1 = Game(dice1)

In [3]:
g1.play(9)
g1.last_play()

,dice-0,dice-1,dice-2
roll,,,
0,3,2,1
1,2,3,1
2,3,2,2
3,3,1,3
4,3,2,1
5,2,2,3
6,1,1,3
7,2,3,3
8,3,1,2


In [4]:
a = Analyzer(g1)
a.permutations()

count
dice-0 dice-1 dice-2       
3      2      1           2
1      1      3           1
2      2      3           1
       3      1           1
              3           1
3      1      2           1
              3           1
       2      2           1